In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# metadata
Experiment = "test_data"
Animals = ["SP111", "SP112"]
Group = ["Control", "Control"]
Path_To_Raw_Data = "/home/sthitapati/Documents/sequence_data/bpod_raw_data"
Camera_Folder = "/home/sthitapati/Documents/sequence_data/SP_FlyCap"
Output_Folder = "/home/sthitapati/Documents/sequence_data/output"

# here the optostim sessions and sessions to discard will also be provided

In [3]:
# Construct the path for the current animal's data
current_animal_id = Animals[1]

# list all directories inside Output_Folder
sessions = os.listdir(Output_Folder + '/' + current_animal_id + '/Preprocessed')
print(sessions)


['45_20230726_201714_Wed', '22_20230531_125541_Wed', '34_20230707_161125_Fri', '38_20230713_163230_Thu', '35_20230710_132549_Mon', '00_20230421_115325_Fri', '31_20230704_160836_Tue', '37_20230712_151830_Wed', '28_20230628_125325_Wed', '15_20230516_155109_Tue', '17_20230518_164704_Thu', '02_20230423_184215_Sun', '21_20230530_132354_Tue', '13_20230511_142820_Thu', '16_20230517_104746_Wed', '09_20230503_121847_Wed', '18_20230519_154229_Fri', '03_20230424_135300_Mon', '36_20230711_170037_Tue', '40_20230717_134402_Mon', '06_20230427_160348_Thu', '23_20230602_135027_Fri', '41_20230718_144029_Tue', '07_20230428_120656_Fri', '30_20230630_120245_Fri', '26_20230627_155644_Tue', '14_20230512_125511_Fri', '24_20230602_143106_Fri', '43_20230721_165344_Fri', '19_20230524_140913_Wed', '12_20230510_130354_Wed', '27_20230627_170858_Tue', '08_20230502_121112_Tue', '04_20230425_134800_Tue', '32_20230705_152522_Wed', '11_20230509_115516_Tue', '10_20230504_134002_Thu', '29_20230629_171114_Thu', '20_2023052

In [4]:
# exctract sessionid and datetime from filename in session

# Initializing empty lists to store extracted information
session_id_list = []
date_list = []
file_number_list = []
day_list = []

# Loop through each file name and extract the information
for session in sessions:
    parts = session.split('_')
    session_id = int(parts[0])  # Assuming you want the session ID as an integer
    date_str = parts[1]
    file_number = int(parts[2])  # Assuming you want the file number as an integer
    day = parts[3]
    
    # Convert date_str to datetime format
    date_obj = datetime.strptime(date_str, "%Y%m%d")
    
    # Append the extracted information to respective lists
    session_id_list.append(session_id)
    date_list.append(date_obj)
    file_number_list.append(file_number)
    day_list.append(day)

In [5]:
# print(session_id_list)
# print(date_list)
# print(file_number_list)
# print(day_list)

# make a dataframe with the sessions, session_id_list, date_list, and day_list and sort it by session_id_list

sessions_df = pd.DataFrame({'session_id': session_id_list, 'session': sessions, 'date': date_list, 'day': day_list})
sessions_df = sessions_df.sort_values(by=['session_id'])
sessions_df = sessions_df.reset_index(drop=True)
sessions_df

,session_id,session,date,day
0,0,00_20230421_115325_Fri,2023-04-21,Fri
1,1,01_20230422_151343_Sat,2023-04-22,Sat
2,2,02_20230423_184215_Sun,2023-04-23,Sun
3,3,03_20230424_135300_Mon,2023-04-24,Mon
4,4,04_20230425_134800_Tue,2023-04-25,Tue
5,5,05_20230426_113720_Wed,2023-04-26,Wed
6,6,06_20230427_160348_Thu,2023-04-27,Thu
7,7,07_20230428_120656_Fri,2023-04-28,Fri
8,8,08_20230502_121112_Tue,2023-05-02,Tue
9,9,09_20230503_121847_Wed,2023-05-03,Wed
